## Brief Description
#### What is it?
Bokeh is an open-souce Python library for making web-interactive visualizations, including 
plots, streaming plots, and dashboards. Bokeh visualizations and dashboards can be easily embedded into webpages,
creating smooth and quick-loading interactions that is unmatched by other interactive visualization libraries 
such as Plotly. 

Bokeh creates shareable, interactive data applications for modern browsers, connecting versatile graphics to PyData tools and to streaming or large datasets, all without having to know JS.  

#### Where is it? Link to project webpage, github
* Here is Bokeh's documentation website: https://docs.bokeh.org/en/latest/
* Here is Bokeh's Github Repository: https://github.com/bokeh/bokeh
- Who developed?
* Bokeh is open-source, and therefore its code is derived widely and from many communities. 
However there are "dedicated core developers" that work on Bokeh. About 2-3 core developers 
dedicate most of their time to Bokeh at a given time. 

#### Why was it created?
 Bokeh was created to help people create interactive visualizations in web browsers, including in Jupyter notebooks. 
It's especially useful to connect PyData tools (e.g. NumPy, SciPy, Pandas, sklearn, etc) to scalable and deployable web "data apps" with little mucking around in "web tech". 
This allows people to add interactive elements to websites while working only in Python. 

#### Advantages & Disadvantages? 
### Bokeh vs. Plotly 
#### Advantages of Bokeh:
 * keeps it simple, no compatibility with other languages = streamlined environment  
 * privacy control 
 * more color palette customization 
 * dynamic and faster Dashboards
 * Bokeh offers an option to embed our visualization in the web pages.
 #### Disadvantages of Bokeh: 
 - only in Python, no compatilbility with other languages
 - no 3D plotting 
 - degree of interactivity is limited 


Potential Use in Environmental Data Science

Quick tutorial/example, using Env. Data Sci example if possible!
Okay to incorporate examples found elsewhere (provide link and acknowledgement!)